In [1]:
import gzip, json
def parse(path):
    with open(path, 'r') as g:
        line = g.readline()
        while line:
            yield json.loads(line)
            line = g.readline()        

In [2]:
fpath = "data/Video_Games_5.json.gz"
next(parse(fpath))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [ ]:
dlist = list()
for i, l in enumerate(parse(fpath)):
    dlist.append(l)
    if i+1 >= 100:
        break

In [ ]:
import pandas as pd
df = pd.DataFrame(dlist)
df.head()

## Add Schema

In [ ]:
import weaviate as wv

In [ ]:
client = wv.Client("http://localhost:8080/")
client.schema.delete_all()
client.schema.get()

In [ ]:
df.info()

In [ ]:
for i, j in df.dtypes.iteritems():
    print(i, j)

In [ ]:
def get_wv_dtype(data_type):
    if data_type == object:
        return "string"
    elif data_type == int:
        return "int"
    elif data_type == bool:
        return "boolean"
    elif data_type == float:
        return "number"
    else:
        return "string"
    

def col_to_prop(name_col, dtype_in):
    prop = {
        "dataType": [get_wv_dtype(dtype_in)],
        "name": name_col,
        "description": f"Contains_{name_col}"
    }
    return prop
props = [
    col_to_prop(colname, datatype) for colname, datatype in df.dtypes.iteritems()
]
props

In [ ]:
class_obj = {
    "class": "Review",
    "description": "An Amazon review entry",
    "properties": props,
}

client.schema.create_class(class_obj)

In [ ]:
client.schema.get()

## Add data

In [ ]:
client.batch.configure(
    batch_size=100,
    dynamic=False,
    timeout_retries=3,
)

In [ ]:
client.batch.add_data_object?

In [ ]:
for i, l in enumerate(parse(fpath)):
    client.batch.add_data_object(
        data_object=l,
        class_name="Review",
    )
    if i+1 >= 100:
        break
client.batch.flush()

In [ ]:
client.data_object.get()

In [ ]:
schema = client.schema.get()
schema['classes'][0]['properties']

## To Func

In [ ]:
import gzip, json
import pandas as pd


def parse(path):
    with gzip.open(path, 'r') as g:
        line = g.readline()
        while line:
            yield json.loads(line)
            line = g.readline()        


# Build DataFrame for schema building
dlist = list()
for i, l in enumerate(parse(fpath)):
    dlist.append(l)
    if i+1 >= 100:
        break

df = pd.DataFrame(dlist)
df.head()


## Add Schema
import weaviate as wv
client = wv.Client("http://localhost:8080/")
client.schema.delete_all()
client.schema.get()


def get_wv_dtype(data_type):
    if data_type == object:
        return "string"
    elif data_type == int:
        return "int"
    elif data_type == bool:
        return "boolean"
    elif data_type == float:
        return "number"
    else:
        return "string"
    

def col_to_prop(name_col, dtype_in):
    prop = {
        "dataType": [get_wv_dtype(dtype_in)],
        "name": name_col,
        "description": f"Contains_{name_col}"
    }
    return prop


props = [
    col_to_prop(colname, datatype) for colname, datatype in df.dtypes.iteritems()
]


class_obj = {
    "class": "Review",
    "description": "An Amazon review entry",
    "properties": props,
}

client.schema.create_class(class_obj)

## Add data
client.batch.configure(
    batch_size=100,
    dynamic=False,
    timeout_retries=3,
)

for i, l in enumerate(parse(fpath)):
    client.batch.add_data_object(
        data_object=l,
        class_name="Review",
    )
    if i+1 >= 100:
        break
client.batch.flush()